In [1]:
import numpy as np
import pandas as pd
import requests

from plotnine import *

# source: https://gist.github.com/rogerallen/1583593
us_state_abbrev = requests.get('https://raw.githubusercontent.com/jwhendy/covid19/master/lib/state_abbrevs.json').json()
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

pop = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/POP01.xls')
pop = pop[['Area_name', 'STCOU', 'POP010210D']]
land = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls')
land = land[['Areaname', 'STCOU', 'LND110210D']]
age = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/AGE01.xls',
                    sheet_name='Sheet2')
age = age[['Areaname', 'STCOU', 'AGE050210D']]
inc = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/INC01.xls')
inc = inc[['Area_name', 'STCOU', 'INC110209D']]

In [64]:
age.head()

,Areaname,STCOU,AGE050210D
0,UNITED STATES,0,37.2
1,ALABAMA,1000,37.9
2,"Autauga, AL",1001,37.0
3,"Baldwin, AL",1003,41.1
4,"Barbour, AL",1005,39.0


In [49]:
### population data: saves out state, county, fips, and population
df = pop.copy()
df.columns = ['area', 'fips', 'pop']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'pop']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/population.csv', index=False)
df.head()

,state,county,fips,pop
2,Alabama,Autauga,1001,54571
3,Alabama,Baldwin,1003,182265
4,Alabama,Barbour,1005,27457
5,Alabama,Bibb,1007,22915
6,Alabama,Blount,1009,57322


In [50]:
### land area: saves out state, county, fips, and land in square miles
df = land.copy()
df.columns = ['area', 'fips', 'land_sqm']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'land_sqm']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/land_sqm.csv', index=False)
df.head()

,state,county,fips,land_sqm
2,Alabama,Autauga,1001,594.44
3,Alabama,Baldwin,1003,1589.78
4,Alabama,Barbour,1005,884.88
5,Alabama,Bibb,1007,622.58
6,Alabama,Blount,1009,644.78


In [62]:
### age: saves out state, county, fips, and median age
df = age.copy()
df.columns = ['area', 'fips', 'age_med']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'age_med']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/age_median.csv', index=False)
df.head()

,state,county,fips,age_med
2,Alabama,Autauga,1001,37.0
3,Alabama,Baldwin,1003,41.1
4,Alabama,Barbour,1005,39.0
5,Alabama,Bibb,1007,37.8
6,Alabama,Blount,1009,39.0


In [63]:
### income: saves out state, county, fips, and median income 2005-2009
df = inc.copy()
df.columns = ['area', 'fips', 'inc_med']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'inc_med']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/income_median.csv', index=False)
df.head()

In [5]:
### pulls all mobility data from google
# - google: https://www.google.com/covid19/mobility/
# - quasi api used below: https://github.com/datasciencecampus/mobility-report-data-extractor
import datetime
import pandas as pd
import os
import re
import subprocess
import time

#data_date = '2020-04-05'
#data_date = '2020-03-29'
seg_list = [x for _ in range(2)
            for x in ['Retail & recreation', 'Grocery & pharmacy', 'Parks',
                      'Transit stations', 'Workplace', 'Residential']]
path = '/home/jwhendy/vault/personal/covid19/'
dir_mob = 'mobility-report-data-extractor'
### run to re-download and process reports
subprocess.call(['./lib/mobility-script.sh'])
areas = [d for d in os.listdir(os.path.join(path, dir_mob, 'output'))
         if d.startswith('US') and data_date in d]

In [5]:
start = time.time()
data_all = []
for area in areas:
    ### re-process pdfs to text
    f = os.path.join(path, dir_mob, 'pdfs', area)
    #subprocess.call(['/usr/bin/pdftotext', '-layout', '-raw', f'{f}.pdf', f'{f}.txt'])
    with open(f'{f}.txt') as f:
        lines = [l for l in f.read().split('\n') if l.strip()]
    #print(lines)
    
    header = re.split(', | ', lines[1])
    date = f'{header[-1]}-{header[-3]}-{header[-2]}'
    date = datetime.datetime.strptime(date, '%Y-%B-%d').strftime('%Y-%m-%d')
    area = ' '.join(header[:-3])
    
    data = []
    for i, line in enumerate(lines):
        if re.findall('Retail & recreation', line) and i<20:
            vals = [re.sub('%|\+', '', lines[i+x]) for x in [1, 13, 26, 38, 49, 59]]
            rows = [{'state': area, 'county': 'summary', 'seg': seg_list[i], 'conf': None, 'value': vals[i]} for i in range(6)]
            data.extend(rows)
        if re.findall('\f', line) and i>50:
            locs = [x.strip() for x in [lines[i], lines[i+13]] for _ in range(6)]
            locs = [l for l in locs if len(l.split(' ')) < 4]
            asts = [lines[i+n-1] for n, x in enumerate(lines[i:i+110]) if x.startswith('Sun')]
            asts = [0 if ast=='*' else 1 for ast in asts]
            vals = [re.sub('%|\+|compared to baseline', '', lines[i+x])
                    for x in [2, 4, 6, 8, 10, 12, 15, 17, 19, 21, 23, 25]]
            vals = [val.strip(' ') if val != 'Not enough data for this date' else None for val in vals]
            segs = [lines[i+n+1] for n in [0, 2, 4, 6, 8, 10, 13, 15, 17, 19, 21, 23]]
            for i, loc in enumerate(locs):
                if segs[i] not in seg_list:
                    continue
                data.append({'state': area, 'county': locs[i], 'seg': segs[i], 'conf': asts[i], 'value': vals[i]})

    for i, d in enumerate(data):
        seq = (6*int(i/6))+(i%6)+1
        data[i]['i'] = seq
        data[i]['path'] = f"output/US-{d['state'].replace(' ', '_', -1)}_{data_date}/csv/{seq}.csv"
    
    data_all.extend(data)
end = time.time()
print(end-start)

df = pd.DataFrame(data_all)
df['county'] = df['county'].str.split(' County', expand=True)[0]
df = df[df['county'] != 'summary']
df = df[df['state'] != 'United States']
df['value'] = pd.to_numeric(df['value'])
df['conf'] = pd.to_numeric(df['conf'])
df.head()

0.3259866237640381


,state,county,seg,conf,value,i,path
6,Alabama,Autauga,Retail & recreation,1.0,-42.0,7,output/US-Alabama_2020-03-29/csv/7.csv
7,Alabama,Autauga,Grocery & pharmacy,1.0,-8.0,8,output/US-Alabama_2020-03-29/csv/8.csv
8,Alabama,Autauga,Parks,0.0,-14.0,9,output/US-Alabama_2020-03-29/csv/9.csv
9,Alabama,Autauga,Transit stations,0.0,NaN,10,output/US-Alabama_2020-03-29/csv/10.csv
10,Alabama,Autauga,Workplace,1.0,-35.0,11,output/US-Alabama_2020-03-29/csv/11.csv


In [51]:
df_save = df.copy()#[['state', 'county', 'seg', 'conf', 'value']]
df_save.to_csv(f'data_raw/mobility-data-agg-raw_{date}.csv', index=False)
#pd.set_option('display.width', 1000)
#print(df_save[(df_save.state=='Ohio') & (df_save.county=='Lucas')])
df[(df.state=='Colorado') & (df.county=='Adams')]

df1 = pd.read_csv('data_raw/mobility-data-agg-raw_2020-03-29.csv')
df2 = pd.read_csv('data_raw/mobility-data-agg-raw_2020-04-05.csv')
df_ts1 = pd.read_csv('data/mobility-data-ts_2020-03-29.csv')
df_ts2 = pd.read_csv('data/mobility-data-ts_2020-04-05.csv')

In [ ]:
ts_all = []
for i, row in df2.iterrows():
    ts = pd.read_csv(os.path.join(path, dir_mob, row.path))[['value', 'date']]
    ts['seg'] = [row.seg]*len(ts)
    ts['state'] = [row['state']]*len(ts)
    ts['county'] = [row['county']]*len(ts)
    ts_all.append(ts)

df_ts = pd.concat(ts_all)
df_ts = df_ts[['state', 'county', 'seg', 'date', 'value']]
df_ts

In [43]:
#df_ts[(df_ts.state=='Ohio') & (df_ts.county=='Mahoning')]
#print(df_ts[(df_ts.state=='Ohio') & (df_ts.county=='Lucas') & (df_ts.seg=='Parks')])
df_ts.to_csv(f'data/mobility-data-ts_{data_date}.csv', index=False)

In [11]:
#df_ts1 = df_ts.copy()
#df_ts2 = df_ts.copy()
#df_ts3 = df_ts1.append(df_ts2[df_ts2.date > df_ts1.date.max()])
#df_ts4 = df_ts1[df_ts1.date < df_ts2.date.min()].append(df_ts2)
#df_ts3.sort_values(['state', 'county', 'date'])
#df_ts4.sort_values(['state', 'county', 'date'])
#df_ts4.to_csv(f'data/mobility-data-ts_all.csv', index=False)
df_ts[(df_ts.state=='Colorado') & (df_ts.county=='Adams')]

,state,county,seg,date,value
0,Colorado,Adams,Retail & recreation,2020-02-16,3.640
1,Colorado,Adams,Retail & recreation,2020-02-17,4.799
2,Colorado,Adams,Retail & recreation,2020-02-18,-2.455
3,Colorado,Adams,Retail & recreation,2020-02-19,-10.355
4,Colorado,Adams,Retail & recreation,2020-02-20,-0.648
...,...,...,...,...,...
38,Colorado,Adams,Residential,2020-03-25,17.812
39,Colorado,Adams,Residential,2020-03-26,21.567
40,Colorado,Adams,Residential,2020-03-27,24.717
41,Colorado,Adams,Residential,2020-03-28,17.595


In [43]:
pd.set_option('display.width', 1000)
print(df1[df1.county=='Solano'])

           state  county                  seg  conf  value    i                                         path
1254  California  Solano  Retail & recreation   1.0  -45.0  271  output/US-California_2020-03-29/csv/271.csv
1255  California  Solano   Grocery & pharmacy   1.0  -21.0  272  output/US-California_2020-03-29/csv/272.csv
1256  California  Solano                Parks   1.0  -18.0  273  output/US-California_2020-03-29/csv/273.csv
1257  California  Solano     Transit stations   1.0  -37.0  274  output/US-California_2020-03-29/csv/274.csv
1258  California  Solano            Workplace   1.0  -34.0  275  output/US-California_2020-03-29/csv/275.csv
1259  California  Solano          Residential   1.0   13.0  276  output/US-California_2020-03-29/csv/276.csv


In [57]:
#print(df_save.head())
#print(df_ts.head())
df_ts_agg = df_ts1.groupby(['state', 'county', 'seg'], as_index=False).agg({'date': 'last', 'value': 'last'})
df_m = df1[['state', 'county', 'seg', 'conf', 'value']].merge(df_ts_agg, on=['state', 'county', 'seg'])
#df_m = df_m[-df_m['value_x'].isnull()]
#df_m = df_m[-df_m['value_y'].isnull()]
#df_m = df_m[df_m['conf']==1]
df_m['delta'] = abs(df_m['value_x'] - df_m['value_y'])
#df_m[df_m['delta'] > 5].to_csv('mobility-errata_2020-03-29.csv', index=False)
df_m

,state,county,seg,conf,value_x,date,value_y,delta
0,Alabama,Autauga,Retail & recreation,1.0,-42.0,2020-03-29,-41.652,0.348
1,Alabama,Autauga,Grocery & pharmacy,1.0,-8.0,2020-03-29,-7.633,0.367
2,Alabama,Autauga,Parks,0.0,-14.0,2020-03-29,-13.953,0.047
3,Alabama,Autauga,Transit stations,0.0,NaN,2020-03-29,NaN,NaN
4,Alabama,Autauga,Workplace,1.0,-35.0,2020-03-29,-35.132,0.132
...,...,...,...,...,...,...,...,...
16789,Wyoming,Weston,Grocery & pharmacy,0.0,-24.0,2020-03-29,-20.831,3.169
16790,Wyoming,Weston,Parks,0.0,NaN,2020-03-29,NaN,NaN
16791,Wyoming,Weston,Transit stations,0.0,NaN,2020-03-29,NaN,NaN
16792,Wyoming,Weston,Workplace,0.0,-34.0,2020-03-29,-34.113,0.113


In [74]:
sah = pd.read_csv('data/sah_dates.csv')
sah.sort_values('date')

,state,date
38,Puerto Rico,2020-03-15
4,California,2020-03-19
27,New Jersey,2020-03-21
14,Illinois,2020-03-21
29,New York,2020-03-22
47,Washington,2020-03-23
36,Oregon,2020-03-23
6,Connecticut,2020-03-23
34,Ohio,2020-03-23
18,Louisiana,2020-03-23


In [89]:
df_ts_sub = df_ts2[df_ts2.county.isin(df[df.conf==1].county.unique())]
p = ggplot(df_ts_sub, aes(x='date', y='value', group='state+county')) + geom_line(alpha=0.05, size=0.1) + facet_wrap('~seg', nrow=3)
p = p + scale_x_datetime()
p = p + theme_bw() + theme(axis_text_x = element_text(angle=315, hjust=0))

In [90]:
p.save('mobility-by-segment_2020-04-05.png', dpi=150, width=10, height=6)

/home/jwhendy/.local/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 10 x 6 in image.
  warn("Saving {0} x {1} {2} image.".format(
/home/jwhendy/.local/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: mobility-by-segment_2020-04-05.png
  warn('Filename: {}'.format(filename), PlotnineWarning)


In [94]:
df_ts_sub = df_ts4[df_ts4.county.isin(df[df.conf==1].county.unique())]
df_ts_sub = df_ts_sub[df_ts_sub['state'].isin(['California', 'New Jersey', 'Florida', 'Georgia'])]
p = ggplot(df_ts_sub, aes(x='date', y='value', group='state+county')) + geom_line(alpha=0.05, size=0.3) + facet_wrap('~state+seg', nrow=4)
p = p + scale_x_datetime()
p = p + theme_bw() + theme(axis_text_x = element_text(angle=315, hjust=0))
p.save('mobility-by-segment_ca-nj-vs-fl-ga_all_2020-04-05.png', dpi=150, width=12, height=6)
#p

/home/jwhendy/.local/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 12 x 6 in image.
  warn("Saving {0} x {1} {2} image.".format(
/home/jwhendy/.local/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: mobility-by-segment_ca-nj-vs-fl-ga_all_2020-04-05.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
